In [ ]:
import redis
import pandas as pd
import time 
import datetime 
import numpy as np

### Ejercicio 1

In [ ]:
class MicroBlog:
    
    # Constructor
    def __init__(self, direccion='127.0.0.1', puerto=6379, base_datos=0):
        self.__r = redis.Redis(host=direccion,port=puerto, password='',db = base_datos,decode_responses=True)
        self.__r.flushall()
        
    
    #---------------- Métodos Públicos ----------------
    
  
    
    def nuevoUsuario(self, nombre_usuario):
        
        # Incrementa el id
        user_id = self.__r.incr('user_id')
        
        # Crea una entrada nueva en la base de datos
        self.__r.set('user:' + str(user_id), nombre_usuario)
        
        # Mete el nombre y el id en una nueva entrada del hash users
        self.__r.hset('users', nombre_usuario, user_id)
        
        
    def seguir(self, seguidor, seguido, fecha):
        
        if (self.__existeUsuario(seguidor)) and (self.__existeUsuario(seguido)):
            self.__nuevoFollower(seguidor, seguido, fecha)
            self.__nuevoFollowing(seguidor, seguido, fecha)       
   
        
        
    def nuevoPost(self, usuario, mensaje, fecha):
        
        # Comprueba que existe el usuario
        if self.__existeUsuario(usuario):
        
            # Incrementa el id
            post_id = self.__r.incr('post_id')
    
            # Obtiene el id del usuario que escribió el mensaje
            user_id = self.__r.hget('users',usuario)
    
            # Crea un hash para el post con el id del usuario, la fecha y el mensaje
            self.__r.hset('post:'+str(post_id), 'user_id', user_id)
            self.__r.hset('post:'+str(post_id), 'fecha', fecha)
            self.__r.hset('post:'+str(post_id), 'mensaje', mensaje)
    
            # Añade el id_post al set de posts del usuario
            self.__r.sadd('posts:'+str(user_id), post_id)
    
            # Obtiene todos los followers del usuario
            lista_followers = self.__r.hgetall('followers:' + user_id).keys()
    
            # Añade el id_post al set de posts de los followers
            for follower_id in lista_followers:
                
                fecha_follow = int(self.__r.hget('followers:' + str(user_id), follower_id))
                
                if fecha >= fecha_follow:                    
                    self.__r.sadd('posts:'+str(follower_id), post_id)   
                
                
    
    def obtenerFollowers(self, usuario):
        
        # Comprueba que existe el usuario
        if self.__existeUsuario(usuario):
            
            # Obtiene el id del usuario al que se va a seguir
            user_id = self.__r.hget('users',usuario)
            
            # Obtiene la lista de ids de los followers
            followers_ids = self.__r.hgetall('followers:' + str(user_id))
            
            # Obtiene los nombres de todos los followers a partir de sus ids
            user_names = []
            for ids in followers_ids.keys(): 
                user_names.append(self.__r.get('user:' + ids))
              
            # Convierte las fechas de formato unix timestamp a datetime
            fechas = []
            for unix in followers_ids.values(): 
                fechas.append(datetime.datetime.utcfromtimestamp(int(unix)).strftime('%Y-%m-%d %H:%M:%S'))
                
            # Une la lista de usuarios y le de fechas en un array 2D
            lista_final = []
            for i in range(len(user_names)):
                lista_final.append([user_names[i],fechas[i]])
                
                
            return lista_final
            
        
    def obtenerFollowings(self, usuario):
        
        # Comprueba que existe el usuario
        if self.__existeUsuario(usuario):
            
            # Obtiene el id del usuario al que se va a seguir
            user_id = self.__r.hget('users',usuario)
            
            # Obtiene la lista de ids de los followers
            followers_ids = self.__r.hgetall('following:' + str(user_id))
            
            # Obtiene los nombres de todos los followers a partir de sus ids
            user_names = []
            for ids in followers_ids.keys(): 
                user_names.append(self.__r.get('user:' + ids))
              
            # Convierte las fechas de formato unix timestamp a datetime
            fechas = []
            for unix in followers_ids.values(): 
                fechas.append(datetime.datetime.utcfromtimestamp(int(unix)).strftime('%Y-%m-%d %H:%M:%S'))
                
        
            # Une la lista de usuarios y le de fechas en un array 2D
            lista_final = []
            for i in range(len(user_names)):
                lista_final.append([user_names[i],fechas[i]])
                
                
            return lista_final
        
        
    def obtenerTimeline(self, usuario, tweets_propios):
        
        # Comprueba que existe el usuario
        if self.__existeUsuario(usuario):
                
            # Obtiene el id del usuario
            user_id = self.__r.hget('users',usuario)
                 
            # Obtiene todos los ids de posts de la timeline del usuario ordenados por fecha descendente
            list_post_ids = self.__r.sort('posts:'+str(user_id), by='post:*->fecha')
            
            # Si tweets_propios es falso, solo muestra los posts de la gente que sigue
            if not tweets_propios:
                
                ids_a_quitar = []
                # Encuentra los post creados por este mismo usuario
                for post_id in list_post_ids:
                    if user_id == self.__r.hget('post:'+str(post_id), 'user_id'):
                        ids_a_quitar.append(post_id)
            
                # Quita los posts del propio usuario
                for post_id in ids_a_quitar:
                    list_post_ids.remove(post_id)
            
            posts = []
            # Obtiene los mensajes de los posts a partir de su id
            for post_id in list_post_ids:
                posts.append(self.__r.hget('post:'+str(post_id), 'mensaje'))
            
            return posts
                
                
                
    
    #---------------- Métodos Privados ----------------
    
    
    
    
    def __nuevoFollower(self, seguidor, usuario, fecha):
        
        # Obtiene el id del usuario al que se va a seguir
        user_id = self.__r.hget('users',usuario)
        
        # Obtiene el id del seguidor
        follower_id = self.__r.hget('users',seguidor)
        
        # Crea una entrada en el hash con los datos del seguidor y la fecha
        self.__r.hset('followers:' + str(user_id), follower_id, fecha)
        
    def __nuevoFollowing(self, usuario, seguido, fecha):
        
        # Obtiene el id del usuario al que empieza a seguir
        user_id = self.__r.hget('users',usuario)
        
        # Obtiene el id del usuario al que se va a seguir
        following_id = self.__r.hget('users',seguido)
        
        # Crea una entrada en el hash con los datos del seguido y la fecha
        self.__r.hset('following:' + str(user_id), following_id, fecha)
    
    def __existeUsuario(self, nombre_usuario):
        return self.__r.hget('users',nombre_usuario) != None


### Ejercicio 2

In [ ]:
df_rel = pd.read_csv('relations.csv')

In [ ]:
# Convierte la columna Following_Time a formato datetime
df_rel['Following_Time'] = pd.to_datetime(df_rel['Following_Time'])

# Crea una nueva columna con la fecha en formato unix timestamp
df_rel['Unix_Following_Time'] = (df_rel['Following_Time'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [ ]:
df_rel.head()

In [ ]:
# Crea una instancia de la clase MicroBlog
twotter = MicroBlog()

In [ ]:
# Añade todos los usuarios a la base de datos con el método nuevoUsuario
lista_usuarios = df_rel['User'].unique()

for usuario in lista_usuarios:
    twotter.nuevoUsuario(usuario)    

In [ ]:
# Añade todos los follows a la base de datos con el método seguir
lista_follows = df_rel[['User','Follows','Unix_Following_Time']].values

for follow in lista_follows:
    twotter.seguir(follow[0], follow[1], follow[2])

In [ ]:
df_twits = pd.read_csv('twitter_sample.csv')
df_twits.drop(columns = ['Unnamed: 3'],inplace=True)

In [ ]:
# Convierte la columna Following_Time a formato datetime
df_twits['Post_Time'] = pd.to_datetime(df_twits['Post_Time'])

# Crea una nueva columna con la fecha en formato unix timestamp
df_twits['Unix_Post_Time'] = (df_twits['Post_Time'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [ ]:
df_twits.head()

In [ ]:
# Introduce todos los tuits en la base de datos con el método nuevoPost
lista_twits = df_twits[['User','Tweet_Content','Unix_Post_Time']].values

for twit in lista_twits:
    twotter.nuevoPost(twit[0], twit[1], twit[2])

### Ejercicio 3

In [ ]:
# Prueba de obtenerFollowers
twotter.obtenerFollowers('andyglittle')

In [ ]:
# Prueba de obtenerFollowings
twotter.obtenerFollowings('andyglittle')

In [ ]:
# Prueba de obtenerTimeline
timeline = twotter.obtenerTimeline('andyglittle', False)

In [ ]:
for twit in timeline:
    print(twit+'\n')